In [1]:
from azureml.core import Workspace, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun, Module, dsl

### Prepare your workspace

In [2]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


### Choose a compute target

In [3]:
aml_compute_name = 'aml-compute'
try:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
except:
    print("Creating new compute target: {}".format(aml_compute_name))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target: aml-compute


### Load data

In [4]:
dataset_name = "THUCNews_For_Batch_Inference"
dataset_score  = workspace.datasets[dataset_name]

### Load module

In [5]:
fasttext_score_module_func = Module.from_yaml(workspace, 'fasttext_score/fasttext_score.spec.yaml')

### Load model

In [6]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['fasttext_test',
 'sample-pipelines',
 'automobile',
 'fasttext_predict',
 'sample-pipelines2',
 'fasttext_with_two_training_process',
 'train-within-notebook',
 'train-on-local',
 'logging-api-test',
 'fasttext_with_one_training_process',
 'fasttext_train',
 'my_test',
 'split_data_txt',
 'compare_two_models',
 'yucongj-test',
 'fasttext_parallel_score',
 'parallel',
 'dir',
 'test0717',
 'test_0727',
 'test_0727_experiment',
 'localtest',
 'mpi_0729',
 'mpi_0729_experiment',
 'test',
 'para_0729',
 'para_0729_experiment',
 'basic_0721',
 'basic_0721_experiment',
 'deploy',
 'fasttext_training_process',
 'fasttext_batch_inference']

### Choose the experiment you want with its name

In [7]:
experiment_name = "fasttext_training_process"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_training_process,fundamental3,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
# azureml.pipeline.core.run.PipelineRun
run = experiment.get_runs().__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,0f59b0d6-b343-4639-8d75-da4b91230007,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [9]:
run_id = run.id
# azureml.pipeline.wrapper._pipeline_run.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,0f59b0d6-b343-4639-8d75-da4b91230007,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline

In [10]:
step_run_list = pipeline_run.find_step_run(name='FastText Train')
for s in step_run_list:
    print(s,'\n\n')

Run(Experiment: fasttext_training_process,
Id: b297d472-86c4-4132-9c59-fcd3cb16301f,
Type: azureml.StepRun,
Status: Completed) 


Run(Experiment: fasttext_training_process,
Id: 66d76ff2-f95e-4fc4-81bd-006943a9e2ec,
Type: azureml.StepRun,
Status: Completed) 




### Get a StepRun object

In [15]:
# step_run_id will be from the visualization result.
step_run_id = '758ab06c-903a-4164-a8b8-44a89ad88aa2'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,06dcef4a-f9c0-4411-b33b-4f80d77c9aab,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Use the trained model as the input of a new pipeline

In [24]:
# name will be from the visualization result.
# get_port() should supports three kinds of names: (1)the_better_model (2)The better model (3)The_better_model
port = step_run.get_port(name='The_better_model')
data_path = port.get_data_path()
model = Dataset.File.from_files(path=[data_path]).as_named_input('model_for_batch_inference')

### Construct the pipeline

In [25]:
@dsl.pipeline(name='batch inference', description='Batch Inference', default_compute_target=aml_compute.name)
def training_pipeline():
    fasttext_score = fasttext_score_module_func(
        texts_to_score=dataset_score,
        fasttext_model_dir=model
    )
    fasttext_score.runsettings.configure(node_count=2, process_count_per_node=2, mini_batch_size="64")

    return {**fasttext_score.outputs}

In [26]:
# pipeline
pipeline = training_pipeline()
# pipeline.save(experiment_name=experiment_name)

In [27]:
# validate
pipeline.validate()

In [28]:
# pipeline_run
experiment_name = 'fasttext_batch_inference'
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=True)
# pipeline_run.wait_for_completion()

Submitted PipelineRun 5891673a-eba2-4450-b858-dc4228378b6f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/5891673a-eba2-4450-b858-dc4228378b6f?wsid=/subscriptions/4f455bd0-f95a-4b7d-8d08-078611508e0b/resourcegroups/fundamental/workspaces/fundamental3
